In [1]:
rm(list = ls())
library(amap)
library(parallelDist)
library(qlcMatrix)
library(doParallel)
library(foreach)

source("source.R")
source("experiments/syntheticGenerator.R")

#setup parallel backend to use many processors
cores=detectCores()
cl <- makeCluster(cores[1]-1) #not to overload your computer
registerDoParallel(cl)

Loading required package: Matrix

Loading required package: slam

Loading required package: sparsesvd

Loading required package: foreach

Loading required package: iterators

Loading required package: parallel

Loading required package: SCCI

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select


Loading required package: pcalg

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘pcalg’”
Loading required package: Rgraphviz

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘Rgraphviz’”


In [2]:
getPairwiseDistArray = function(data, inds, isCat=c()){
    N = dim(data)[1]
    nDim = dim(data)[2]
    if(length(inds)==0){
        inds = 1:nDim
    }
    disArray = array(rep(NaN, N*N*nDim), dim =c(N,N,nDim))
    for(m in inds){
        dataDim = as.matrix(data[,m])
        if(m %in% isCat){
            for(i in 1:N){
                for(j in 1:N){
                    if(dataDim[i]==dataDim[j]){
                        disArray[i,j,m] = 0
                    }else{
                        disArray[i,j,m] = 1
                    }
                }
            }
        }else{
            dataDim = as.matrix(dataDim, nrow=N)
            disArray[,,m] = as.matrix(parDist(dataDim, method="manhattan"))
        }
    }
    return(disArray)
 }

In [3]:
getPointCoordDists = function(distArray, point_i, inds=c()){
    if(length(inds)==0){
        inds = 1:dim(distArray)[3]
    }
    obsDists =  distArray[,point_i, inds]
    return(obsDists)
}

In [4]:
getKnnDistCont = function(coord_dists, k){
    dists = as.numeric(rowMax(coord_dists, ignore.zero=F))
    ordered_dists = sort(dists)
    return(c(k, ordered_dists[k+1]))
}

getKnnDistRavk = function(coord_dists, k){
    dists = as.numeric(rowMax(coord_dists, ignore.zero=F))
    ordered_dists = sort(dists)
    if(ordered_dists[k+1]==0){
        k_tilde = length(which(dists == 0)) - 1
    }else{
        k_tilde = k
        
    }
    return(c(k_tilde, ordered_dists[k+1]))
}

getKnnDist = function(coord_dists, k){
    dists = as.numeric(rowMax(coord_dists, ignore.zero=F))
    ordered_dists = sort(dists)
    k_tilde = length(which(dists <= ordered_dists[k+1])) - 1
    return(c(k_tilde, ordered_dists[k+1]))
}

In [5]:
countNeighborsCont = function(coord_dists, rho, coords=c()){
    if(length(coords)==0){
        coords = 1:dim(coord_dists)[2]
    }
    dists = rowMax(coord_dists[,coords], ignore.zero=F)
    count = max(length(which(dists < rho)) - 1, 1)
    return(count)
}

countNeighbors= function(coord_dists, rho, coords=c()){
    if(length(coords)==0){
        coords = 1:dim(coord_dists)[2]
    }
    dists = rowMax(coord_dists[,coords], ignore.zero=F)
    count = length(which(dists <= rho)) - 1
    return(count)
}


In [6]:
cmiEstFP = function(coord_dists, k,  x_coords, y_coords, z_coords){
    resKR =  getKnnDistCont(coord_dists=coord_dists, k=k)
    k_tilde = as.integer(resKR[1])
    rho = resKR[2]
    nxz = countNeighborsCont(coord_dists, rho, c(x_coords,z_coords))
    nyz = countNeighborsCont(coord_dists, rho, c(y_coords,z_coords))
    nz = countNeighborsCont(coord_dists, rho, z_coords)
    xiFP = digamma(k_tilde) - digamma(nxz) - digamma(nyz) + digamma(nz)
    return(xiFP)
}

cmiEstRAVK1 = function(coord_dists, k,  x_coords, y_coords, z_coords){
    resKR = getKnnDistRavk(coord_dists=coord_dists, k=k)
    k_tilde = as.integer(resKR[1])
    rho = resKR[2]
    nxz = countNeighbors(coord_dists, rho, c(x_coords,z_coords))
    nyz = countNeighbors(coord_dists, rho, c(y_coords,z_coords))
    nz = countNeighbors(coord_dists, rho, z_coords)
    xiRAVK1 = digamma(k_tilde) - log(nxz + 1) - log(nyz + 1) + log(nz + 1)
    return(xiRAVK1)  
}

cmiEstRAVK2 = function(coord_dists, k,  x_coords, y_coords, z_coords){
    resKR = getKnnDist(coord_dists=coord_dists, k=k)
    k_tilde = as.integer(resKR[1])
    rho = resKR[2]
    nxz = countNeighbors(coord_dists, rho, c(x_coords,z_coords))
    nyz = countNeighbors(coord_dists, rho, c(y_coords,z_coords))
    nz = countNeighbors(coord_dists, rho, z_coords)
    xiRAVK2 = digamma(k_tilde) - log(nxz + 1) - log(nyz + 1) + log(nz + 1)
    return(xiRAVK2)
}

cmiEstMS = function(coord_dists, k,  x_coords, y_coords, z_coords){
    resKR = getKnnDist(coord_dists=coord_dists, k=k)
    k_tilde = as.integer(resKR[1])
    rho = resKR[2]
    nxz = countNeighbors(coord_dists, rho, c(x_coords,z_coords))
    nyz = countNeighbors(coord_dists, rho, c(y_coords,z_coords))
    nz = countNeighbors(coord_dists, rho, z_coords)
    xiMS = digamma(k_tilde) - digamma(nxz) - digamma(nyz) + digamma(nz)
    return(xiMS)
}

In [7]:
KNN.estimates = function(data, xind, yind, zinds, k= 3, isCat=c(), logE=T){
    N = dim(dd)[1]
    inds = c(xind, yind, zinds)
    distArray = getPairwiseDistArray(data=as.matrix(data), inds=inds, isCat=isCat)
    cmiFP = 0
    cmiMS = 0
    cmiRAVK1 = 0
    cmiRAVK2 = 0
    for(i in 1:N){
        coord_dists = getPointCoordDists(distArray=distArray, point_i=i, inds=inds)
        x_coords = 1:length(xind)
        y_coords = (length(xind)+1):length(c(xind, yind))
        z_coords = (length(c(xind,yind))+1):length(c(xind, yind, zinds))
        cmiFP = cmiFP + cmiEstFP(coord_dists=coord_dists, k=k,  x_coords=x_coords, y_coords=y_coords, z_coords=z_coords)
        cmiRAVK1 = cmiRAVK1 + cmiEstRAVK1(coord_dists=coord_dists, k=k,  x_coords=x_coords, y_coords=y_coords, z_coords=z_coords)
        cmiRAVK2 = cmiRAVK2 + cmiEstRAVK2(coord_dists=coord_dists, k=k,  x_coords=x_coords, y_coords=y_coords, z_coords=z_coords)
        cmiMS = cmiMS + cmiEstMS(coord_dists=coord_dists, k=k,  x_coords=x_coords, y_coords=y_coords, z_coords=z_coords)
    }
    return(list(FP=cmiFP/N, GAVK1=cmiRAVK1/N, GAVK2=cmiRAVK2/N, MS=cmiMS/N))
}

KNN.estimates_Parall = function(data, xind, yind, zinds, k= 3, isCat=c(), logE=T){
    N = dim(dd)[1]
    inds = c(xind, yind, zinds)
    distArray = getPairwiseDistArray(data=as.matrix(data), inds=inds, isCat=isCat)
    cmiFP = 0
    cmiMS = 0
    cmiRAVK1 = 0
    cmiRAVK2 = 0
    for(i in 1:N){
        coord_dists = getPointCoordDists(distArray=distArray, point_i=i, inds=inds)
        x_coords = 1:length(xind)
        y_coords = (length(xind)+1):length(c(xind, yind))
        z_coords = (length(c(xind,yind))+1):length(c(xind, yind, zinds))
        cmiFP = cmiFP + cmiEstFP(coord_dists=coord_dists, k=k,  x_coords=x_coords, y_coords=y_coords, z_coords=z_coords)
        cmiRAVK1 = cmiRAVK1 + cmiEstRAVK1(coord_dists=coord_dists, k=k,  x_coords=x_coords, y_coords=y_coords, z_coords=z_coords)
        cmiRAVK2 = cmiRAVK2 + cmiEstRAVK2(coord_dists=coord_dists, k=k,  x_coords=x_coords, y_coords=y_coords, z_coords=z_coords)
        cmiMS = cmiMS + cmiEstMS(coord_dists=coord_dists, k=k,  x_coords=x_coords, y_coords=y_coords, z_coords=z_coords)
    }
    return(c(cmiFP/N, cmiRAVK1/N, cmiRAVK2/N, cmiMS/N))
}

In [ ]:
steps = 1:10
loops = 1:100
fact = 100

mean_vec = data.frame(Samples=steps*fact, FP=rep(0, length(steps)), GAVK1=rep(0, length(steps)), GAVK2=rep(0, length(steps)), MS=rep(0, length(steps)))
sd_vec = data.frame(Samples=steps*fact, FP=rep(0, length(steps)), GAVK1=rep(0, length(steps)), GAVK2=rep(0, length(steps)), MS=rep(0, length(steps)))
meanse_vec = data.frame(Samples=steps*fact, FP=rep(0, length(steps)), GAVK1=rep(0, length(steps)), GAVK2=rep(0, length(steps)), MS=rep(0, length(steps)))

index = 1
m = 5 
trueI= log(m) - (m-1)*log(2)/m

for(s in steps){
    n = s * fact
    estimates = data.frame(FP=rep(0, length(loops)), GAVK1=rep(0, length(loops)), GAVK2=rep(0, length(loops)), MS=rep(0, length(loops)))
    for(l in loops){
        dd = test2(n,m=m)
        estimate = KNN.estimates(data=dd, xind=c(1), yind=c(2), zinds=c(3))
        for(i in 1:4){
          estimates[l,i] = max(estimate[[i]],0)
        }
    }
    for(i in 1:4){
        mean_vec[index,i+1] = mean(estimates[,i])
        sd_vec[index,i+1] = sd(estimates[,i])
        meanse_vec[index,i+1] = sum((estimates[,i] - trueI)^2) / length(loops)
    }
    index = index + 1
}

write.table(mean_vec, file="test2_mean.tab", sep="\t", row.names=F, col.names=T, quote=F)
write.table(sd_vec, file="test2_sd.tab", sep="\t", row.names=F, col.names=T, quote=F)
write.table(meanse_vec, file="test2_meanse.tab", sep="\t", row.names=F, col.names=T, quote=F)

In [ ]:
dataHisto = read.table("results/test1_meanse5.tab", header=T) 
Histo_LH = dataHisto$LH
Histo_Chisq99 = dataHisto$Chisq99

In [ ]:
xLab = seq(100,1000,100)
plot(xLab, meanse_vec[,2],
main="MSE",
#ylab="",
ylim=c(0,0.2),
type="l",
col="#7fc97f")
lines(xLab,meanse_vec[,3], col="#beaed4")
lines(xLab,meanse_vec[,4], col="#fdc086")
lines(xLab,meanse_vec[,5], col="#ff7f00")
lines(xLab,Histo_LH, col="#386cb0")
lines(xLab,Histo_Chisq99, col="#f0027f")
legend("topleft",
c("FP","GAVK1","GAVK2", "MS", "Histo_LH", "Histo_Chisq99"),
fill=c("#7fc97f", "#beaed4", "#fdc086", "#ff7f00", "#386cb0", "#f0027f")
)

In [ ]:
steps = 1:10
loops = 1:100
fact = 100

mean_vec = data.frame(Samples=steps*fact, FP=rep(0, length(steps)), GAVK1=rep(0, length(steps)), GAVK2=rep(0, length(steps)), MS=rep(0, length(steps)))
sd_vec = data.frame(Samples=steps*fact, FP=rep(0, length(steps)), GAVK1=rep(0, length(steps)), GAVK2=rep(0, length(steps)), MS=rep(0, length(steps)))
meanse_vec = data.frame(Samples=steps*fact, FP=rep(0, length(steps)), GAVK1=rep(0, length(steps)), GAVK2=rep(0, length(steps)), MS=rep(0, length(steps)))

index = 1
m = 5 
trueI= log(m) - (m-1)*log(2)/m

for(s in steps){
    n = s * fact
    estimates<-foreach(l = loops, .combine=rbind, .packages = c('parallelDist','qlcMatrix')) %dopar% {
        dd = test2(n,m=m)
        KNN.estimates_Parall(data=dd, xind=c(1), yind=c(2), zinds=c(3))
    }
    for(i in 1:4){
        mean_vec[index,i+1] = mean(estimates[,i])
        sd_vec[index,i+1] = sd(estimates[,i])
        meanse_vec[index,i+1] = sum((estimates[,i] - trueI)^2) / length(loops)
    }
    index = index + 1
}

write.table(mean_vec, file="test2Parral_mean.tab", sep="\t", row.names=F, col.names=T, quote=F)
write.table(sd_vec, file="test2Parral_sd.tab", sep="\t", row.names=F, col.names=T, quote=F)
write.table(meanse_vec, file="test2Parral_meanse.tab", sep="\t", row.names=F, col.names=T, quote=F)

In [ ]:
dataHisto = read.table("results/test1_meanse5.tab", header=T) 
Histo_LH = dataHisto$LH
Histo_Chisq99 = dataHisto$Chisq99

In [ ]:
xLab = seq(100,1000,100)
plot(xLab, meanse_vec[,2],
main="MSE",
#ylab="",
ylim=c(0,0.2),
type="l",
col="#7fc97f")
lines(xLab,meanse_vec[,3], col="#beaed4")
lines(xLab,meanse_vec[,4], col="#fdc086")
lines(xLab,meanse_vec[,5], col="#ff7f00")
lines(xLab,Histo_LH, col="#386cb0")
lines(xLab,Histo_Chisq99, col="#f0027f")
legend("topleft",
c("FP","GAVK1","GAVK2", "MS", "Histo_LH", "Histo_Chisq99"),
fill=c("#7fc97f", "#beaed4", "#fdc086", "#ff7f00", "#386cb0", "#f0027f")
)

In [ ]:
Sys.time()